In [1]:
import torch
from slim_gsgp.main_mo_gp import mo_gp
from slim_gsgp.datasets.data_loader import load_ppb
from slim_gsgp.utils.utils import train_test_split

# General Multi-Objective Settings
MO_FFUNCS = ["rmse", "size"] 
MO_MIN_FLAGS = [True, True]
MO_TS_SIZES = [4, 2]
IDEAL_POINTS = [0.0, 0.0] # For cenario 5

# Load Data
X, y = load_ppb(X_y=True)
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, p_test=0.4, seed=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, p_test=0.5, seed=42)

# Function to run and print
def run_scenario(name, **kwargs):
    print(f"\n{'='*70}\nRunning Scenario: {name}\n{'='*70}")
    
    # Default parameters for quick testing
    default_args = {
        'pop_size': 50,
        'n_iter': 10,
        'n_elites': 1,
        'ideal_candidate_values': None,
        'seed': 42
    }
    params = {**default_args, **kwargs}

    # excute the MOGP
    try:
        elite = mo_gp(
            X_train=X_train, y_train=y_train,
            X_test=X_val, y_test=y_val,
            dataset_name='ppb_test',
            fitness_functions=MO_FFUNCS,
            minimization_flags=MO_MIN_FLAGS,
            tournament_sizes=MO_TS_SIZES,
            verbose=1,
            log_level=0, # change to 1 to save CSVs in the log/ folder
            n_jobs=2,
            **params
        )
        print(f"> Done. Elite Fitness: {elite.fitness}")
    except Exception as e:
        print(f"> Error in scenario '{name}': {e}")
        import traceback
        traceback.print_exc()



# =============================================================================
# 1. NSGA-II Puro
# Parent Selection: NSGA-II | Survival: NSGA-II
# =============================================================================
run_scenario(
    "1. NSGA-II Puro",
    selector_strategy="nsga2",
    survival_strategy="nsga2",
    n_elites=1 # maintains the best ranks
)

# =============================================================================
# 2. Nested Tournament + NSGA-II Survival
# Parent Selection: NT | Survival: NSGA-II
# =============================================================================
run_scenario(
    "2. Nested Parent Selection + NSGA-II Survival",
    selector_strategy="nested_tournament",
    survival_strategy="nsga2"
)

# =============================================================================
# 3. Nested Tournament + Direct Replacement (No Elitism)
# Parent Selection: NT | Survival: Generational (Children replace parents)
# =============================================================================
run_scenario(
    "3. Nested + Direct Replacement (No Elitism)",
    selector_strategy="nested_tournament",
    survival_strategy="generational",
    n_elites=0
)

# =============================================================================
# 4. Nested Tournament + First Objective Elitism
# Parent Selection: NT | Survival: Generational with Elite on 1st Obj
# =============================================================================
run_scenario(
    "4. Nested + Direct Replacement + 1st Obj Elitism",
    selector_strategy="nested_tournament",
    survival_strategy="generational",
    n_elites=2,
    ideal_candidate_values=None
)

# =============================================================================
# 5. Nested Tournament + Ideal Candidate Elitism
# Parent Selection: NT | Survival: Generational with Ideal Candidate Elitism
# =============================================================================
run_scenario(
    "5. Nested + Direct Replacement + Ideal Candidate Elitism",
    selector_strategy="nested_tournament",
    survival_strategy="generational",
    n_elites=2,
    ideal_candidate_values=IDEAL_POINTS
)


Running Scenario: 1. NSGA-II Puro

--------------------------------------------------------------------------------------------------------
| Dataset         |  Gen   | Train Fitness           | Test Fitness            |    Time    |  Nodes   |
--------------------------------------------------------------------------------------------------------
| ppb_test        |   0    | 32.5666 | 3.0000        | 37.6712 | 44.4714       |   3.4851   |    3     |
| ppb_test        |   1    | 32.5305 | 3.0000        | 39.9997 | 65.5645       |   0.2653   |    3     |
| ppb_test        |   2    | 32.5305 | 3.0000        | 39.9997 | 65.5645       |   0.2618   |    3     |
| ppb_test        |   3    | 31.1110 | 3.0000        | 36.6421 | 43.5195       |   0.2624   |    3     |
| ppb_test        |   4    | 31.1110 | 3.0000        | 36.6421 | 43.5195       |   0.2526   |    3     |
| ppb_test        |   5    | 31.1110 | 3.0000        | 36.6421 | 43.5195       |   0.2457   |    3     |
| ppb_test        |

In [ ]:
# !pip install pytest-mock
# !pip install pytest
# import pytest
# pytest.main(["tests/test_mo_gp.py"])
# pytest.main(["tests/test_validate_inputs.py"])
# pytest.main(["tests/test_utils.py"])

============================= test session starts =============================
platform win32 -- Python 3.10.16, pytest-8.3.4, pluggy-1.5.0
rootdir: c:\Users\filip\OneDrive\Documentos\NOVA\Tese\GitHub\slim
plugins: anyio-4.6.2, mock-3.15.1
collected 6 items

tests\test_utils.py ......                                               [100%]

============================== 6 passed in 3.04s ==============================


<ExitCode.OK: 0>

<div style="background-color:#e5c120ff; padding:1px; border-radius:10px;">
</div>